# Topological Index Calculator
This notebook calculates different topological indices (Edge Density, Wiener Index, Petitjean Index) from molecular structures.

In [1]:
import os
import sys


In [2]:
def get_index_choice():
    while True:
        print('Which topological index would you like to calculate?')
        print('1 - Edge Density')
        print('2 - Wiener Index')
        print('3 - Petitjean Index')
        print('4 - All of the above')

        choice = input('Enter your choice (1-4): ').strip()

        options = {
            '1': 'edge',
            '2': 'wiener',
            '3': 'petitjean',
            '4': 'all'
        }

        selected = options.get(choice)
        if selected:
            return selected
        print('Invalid choice. Please select 1, 2, 3, or 4.')
